#### 📚 For more information please visit our [GitHub](https://github.com/tecepeipe/RcloneLab).


# <img src='https://raw.githubusercontent.com/Sparoney/RcloneLab/master/img/title_rclonelab.png' height="45" alt="RcloneLab"/>

In [0]:
#@markdown <h3>⬅️ Click Here to install RcloneLab</h3>
#@markdown <h4>📝 Note: rerun this cell to change the config file.</h4>
Beta = False #@param {type:"boolean"}
Setup_Time_Zone = False #@param {type:"boolean"}

import os; from google.colab import files; from IPython.display import HTML, clear_output

def upload_conf():
  try:
    display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Please upload rclone config file (rclone.conf) from your computer.</h2><br>"))
    UploadConfig = files.upload().keys()
    if len(UploadConfig) == 0:
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
    elif len(UploadConfig) == 1:
      for fn in UploadConfig:
        if os.path.isfile("/content/" + fn):
          get_ipython().system_raw("mv -f " + "\"" + fn + "\" /root/.rclone.conf && chmod 666 /root/.rclone.conf")
          if Setup_Time_Zone:
            !sudo dpkg-reconfigure tzdata
          if os.path.isfile("/usr/bin/rclone"):
            clear_output(wait=True)
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">Config has been changed.</h2><br></center>"))
          else:
            if Beta:
              get_ipython().system_raw("rm -rf /content/sample_data/ && curl -s https://rclone.org/install.sh | sudo bash -s beta")
            else:
              get_ipython().system_raw("rm -rf /content/sample_data/ && curl -s https://rclone.org/install.sh | sudo bash")
            clear_output(wait=True)
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Installation has been successfully completed.</h2><br></center>"))
        else:
          clear_output(wait=True)
          return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
    else:
      for fn in UploadConfig:
        get_ipython().system_raw("rm -f " + "\"" + fn + "\"")
      clear_output(wait=True)
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
  except:
    clear_output(wait=True)
    return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))

upload_conf()

In [0]:
#@markdown <h3>⬅️ Run This Cell to Use RcloneLab</h3>
# ============================= FORM ============================= #
Mode = "Copy" #@param ["Config", "Copy", "Move", "Mount", "Sync", "Checker", "Deduplicate", "Remove Empty Directories", "Empty Trash", "qBittorrent"]
Compare = "Only Size" #@param ["Size & Mod-Time", "Size & Checksum", "Only Mod-Time", "Only Size", "Only Checksum"]
Source = "gdrive-bundle:TV Series" #@param {type:"string"}
Destination = "onedrive-bundle:TV Series" #@param {type:"string"}
Transfers = 5 #@param {type:"slider", min:1, max:20, step:1}
Checkers = 5 #@param {type:"slider", min:1, max:20, step:1}
Trans_per_sec = 5 #@param {type:"slider", min:1, max:20, step:1}
#@markdown ---

#@markdown <center><h3><font color="#3399ff"><b>⚙️ Global Configuration ⚙️</b></font></h3></center>
Dry_Run = False #@param {type:"boolean"}
Stats_one_line = True
Skip_files_that_are_newer_on_the_destination = True #@param {type:"boolean"}
Skip_all_files_that_exist = False #@param {type:"boolean"}
Do_not_update_modtime_if_files_are_identical = True #@param {type:"boolean"}
Large_amount_of_files_optimization = True #@param {type:"boolean"}
Google_Drive_optimization = True #@param {type:"boolean"}
Google_Drive_server_side_copy = False #@param {type:"boolean"}
Do_not_cross_filesystem_boundaries = False
Output_Log_File = "OFF" #@param ["OFF", "NOTICE", "INFO", "ERROR", "DEBUG"]
Extra_Arguments = "--max-size 5G --exclude \"Gamez/**\"" #@param {type:"string"}

#@markdown <center><h3><font color="#3399ff"><b>↪️ Sync Configuration ↩️</b></font></h3></center>
Sync_Mode = "Delete during transfer" #@param ["Delete during transfer", "Delete before transfering", "Delete after transfering"]
Track_Renames = False #@param {type:"boolean"}

#@markdown <center><h3><font color="#3399ff"><b>💞 Deduplicate Configuration 💞</b></font></h3></center>
Deduplicate_Mode = "First" #@param ["Interactive", "Skip", "First", "Newest", "Oldest", "Largest", "Rename"]
Deduplicate_Use_Trash = False #@param {type:"boolean"}
# ================================================================ #

import os, psutil; from IPython.display import HTML, clear_output; from IPython.utils.io import ask_yes_no

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue

### Variable Declaration

if Compare == "Size & Checksum":
  compareC = "-c"
elif Compare == "Only Mod-Time":
  compareC = "--ignore-size"
elif Compare == "Only Size":
  compareC = "--size-only"
elif Compare == "Only Checksum":
  compareC = "-c --ignore-size"
else:
  compareC = ""

transfersC = "--transfers "+str(Transfers)
checkersC = "--checkers "+str(Checkers)
tpslimitC = "--tpslimit "+str(Trans_per_sec)

if Skip_files_that_are_newer_on_the_destination:
  skipnewC = "-u"
else:
  skipnewC = ""
  
if Skip_all_files_that_exist:
  skipexistC = "--ignore-existing"
else:
  skipexistC = ""
  
if Do_not_cross_filesystem_boundaries:
  nocrossfilesystemC = "--one-file-system"
else:
  nocrossfilesystemC = ""
  
if Do_not_update_modtime_if_files_are_identical:
  noupdatemodtimeC = "--no-update-modtime"
else:
  noupdatemodtimeC = ""

if Large_amount_of_files_optimization:
  filesoptimizeC = "--fast-list"
else:
  filesoptimizeC = ""
  
if Google_Drive_optimization:
  driveoptimizeC = "--drive-chunk-size 128M --drive-stop-on-upload-limit --drive-acknowledge-abuse --drive-keep-revision-forever --drive-pacer-min-sleep=0ms"
else:
  driveoptimizeC = ""

if Google_Drive_server_side_copy:
  drivecopyC = "--drive-server-side-across-configs=true"
elif Google_Drive_optimization:
  drivecopyC = "--disable copy"

# Optimized for Google Colaboratory
if Google_Drive_optimization:
  bufferC = "--buffer-size 128M"
else:
  bufferC = "--buffer-size 256M"

if Dry_Run:
  dryrunC = "-n"
else:
  dryrunC = ""
  
if Output_Log_File != "OFF":
  statsC = "--log-file=/root/.rclone_log/rclone_log.txt"
else:
  if Stats_one_line:
    statsC = "-v --stats-one-line --stats=5s"
  else:
    statsC = "-v --stats=5s"
  
if Output_Log_File == "INFO":
  loglevelC = "--log-level INFO"
elif Output_Log_File == "ERROR":
  loglevelC = "--log-level ERROR"
elif Output_Log_File == "DEBUG":
  loglevelC = "--log-level DEBUG"
else:
  loglevelC = ""

if Sync_Mode == "Delete during transfer":
  syncmodeC = "--delete-during"
elif Sync_Mode == "Delete before transfering":
  syncmodeC = "--delete-before"
elif Sync_Mode == "Delete after transfering":
  syncmodeC = "--delete-after"
  
if Track_Renames:
  trackrenamesC = "--track-renames"
else:
  trackrenamesC = ""
  
if Deduplicate_Mode == "Interactive":
  deduplicateC = "interactive"
elif Deduplicate_Mode == "Skip":
  deduplicateC = "skip"
elif Deduplicate_Mode == "First":
  deduplicateC = "first"
elif Deduplicate_Mode == "Newest":
  deduplicateC = "newest"
elif Deduplicate_Mode == "Oldest":
  deduplicateC = "oldest"
elif Deduplicate_Mode == "Largest":
  deduplicateC = "largest"
elif Deduplicate_Mode == "Rename":
  deduplicateC = "rename"
  
if Deduplicate_Use_Trash:
  deduplicatetrashC = ""
else:
  deduplicatetrashC = "--drive-use-trash=false"
  
### rclone Execution

if Output_Log_File != "OFF" and Mode != "Config":
  get_ipython().system_raw("mkdir -p -m 666 /root/.rclone_log/")
  display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Logging enables, rclone will not output log through the terminal, please wait until finished.</h2><br></center>"))

if Mode == "Config":
  !rclone --config=/root/.rclone.conf config
elif Mode == "Copy":
  !rclone --config=/root/.rclone.conf copy "$Source" "$Destination" $transfersC $checkersC $tpslimitC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments
elif Mode == "Move":
  !rclone --config=/root/.rclone.conf move "$Source" "$Destination" $transfersC $checkersC $tpslimitC $statsC $loglevelC --delete-empty-src-dirs $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments
elif Mode == "Mount":
  try:
    if not CheckProcess("rclone", "/content/" + Source.split(":")[0] + "/"):
      if not os.path.exists("/content/" + Source.split(":")[0] + "/"):
        get_ipython().system_raw("mkdir -p -m 666 '/content/" + Source.split(":")[0] + "/'")
      get_ipython().system_raw("rclone --config=/root/.rclone.conf mount '$Source' '/content/" + Source.split(":")[0] + "/' $loglevelC $noupdatemodtimeC $bufferC $filesoptimizeC --drive-chunk-size 128M --drive-acknowledge-abuse --drive-keep-revision-forever $Extra_Arguments &")
  except:
    pass
elif Mode == "Sync":
  !rclone --config=/root/.rclone.conf sync "$Source" "$Destination" $transfersC $checkersC $tpslimitC $statsC $loglevelC $syncmodeC $trackrenamesC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments
elif Mode == "Checker":
  !rclone --config=/root/.rclone.conf check "$Source" "$Destination" $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments
elif Mode == "Deduplicate":
  !rclone --config=/root/.rclone.conf dedupe "$Source" $checkersC $tpslimitC $statsC $loglevelC --dedupe-mode $deduplicateC $deduplicatetrashC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments
elif Mode == "Remove Empty Directories":
  !rclone --config=/root/.rclone.conf rmdirs "$Source" $statsC $loglevelC $dryrunC $Extra_Arguments
elif Mode == "Empty Trash":
  !rclone --config=/root/.rclone.conf cleanup "$Source" $statsC $loglevelC $dryrunC $Extra_Arguments
elif Mode == "qBittorrent":
  if Source == "":
    !chmod -R 666 /content/qBittorrent/ && rclone --config=/root/.rclone.conf move "/content/qBittorrent/" "$Destination" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs --exclude **/.unwanted/ $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments

### Log Output
  
if Output_Log_File != "OFF" and Mode != "Config" and os.path.isfile("/root/.rclone_log/rclone_log.txt"):
  ### Rename log file and output settings.
  get_ipython().system_raw("mv /root/.rclone_log/rclone_log.txt /root/.rclone_log/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).txt")
  with open("/root/.rclone_log/" + Mode + "_settings.txt", "w") as f:
    f.write("Mode: " + Mode + \
            "\nCompare: " + Compare + \
            "\nSource: \"" + Source + \
            "\"\nDestination: \"" + Destination + \
            "\"\nTransfers: " + str(Transfers) + \
            "\nCheckers: " + str(Checkers) + \
            "\nSkip files that are newer on the destination: " + str(Skip_files_that_are_newer_on_the_destination) + \
            "\nSkip all files that exist: " + str(Skip_all_files_that_exist) + \
            "\nDo not cross filesystem boundaries: " + str(Do_not_cross_filesystem_boundaries) + \
            "\nDo not update modtime if files are identical: " + str(Do_not_update_modtime_if_files_are_identical) + \
            "\nDry-Run: " + str(Dry_Run) + \
            "\nOutput Log Level: " + Output_Log_File + \
            "\nExtra Arguments: \"" + Extra_Arguments + \
            "\"\nSync Moden: " + Sync_Mode + \
            "\nTrack Renames: " + str(Track_Renames) + \
            "\nDeduplicate Mode: " + Deduplicate_Mode + \
            "\nDeduplicate Use Trash: " + str(Deduplicate_Use_Trash))
  ### Compressing log file.
  get_ipython().system_raw("rm -f /root/rclone_log.zip && zip -r -q -j -9 /root/rclone_log.zip /root/.rclone_log/ && rm -rf /root/.rclone_log/ && mkdir -p -m 666 /root/.rclone_log/")
  ### Send Log
  if os.path.isfile("/root/rclone_log.zip"):
    try:
      files.download("/root/rclone_log.zip")
      get_ipython().system_raw("rm -f /root/rclone_log.zip")
      display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Sending log to your browser...</h2><br></center>"))
    except:
      get_ipython().system_raw("mv /root/rclone_log.zip /content/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).zip")
      display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">You can use file explorer to download the log file.</h2><br><img src=\"https://raw.githubusercontent.com/Sparoney/RcloneLab/master/res/rclonelab/01.png\"><br></center>"))
  else:
    clear_output(wait=True)
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">There is no log file.</h2><br></center>"))

### Operation has been successfully completed.
if os.path.isfile("/usr/bin/rclone") == False:
  clear_output(wait=True)
  display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ Please run the installation cell above first.</h2><br></center>"))
elif Mode == "qBittorrent" and Source.strip() != "":
  display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ Please empty the Source field in qBittorrent mode.</h2><br></center>"))
elif Mode == "Mount" and Source.strip() == "":
  display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ The Source field is empty.</h2><br></center>"))
elif Mode != "Config":
  display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">✅ Operation has been successfully completed.</h2><br></center>"))

# <img src='https://raw.githubusercontent.com/Sparoney/RcloneLab/master/img/title_qbittorrent.png' height="45" alt="qBittorrent"/>

In [0]:
# ============================= FORM ============================= #
#@markdown <h3>⬅️ Click Here to Install qBittorrent</h3>
#@markdown <h4>❌ Don't use the mounted Google Drive as a save path for torrent; data corruption will most likely occur.</h4>
#@markdown <h4>✅ The rclone method is recommended and always the safest.</h4>
Old_Version = False #@param {type:"boolean"}
# ================================================================ #

import os, psutil, IPython, uuid, time
import ipywidgets as widgets

from IPython.display import HTML, clear_output
from google.colab import output

class MakeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    template = """<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-info" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def RandomGenerator():
  return time.strftime("%S") + str(time.time()).split(".")[-1]

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue
  
def AutoSSH(name,port):
  get_ipython().system_raw("autossh -M 0 -fNT -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R " + name + ":80:localhost:" + port + " serveo.net &")

def Start_AutoSSH_QB():
  if CheckProcess("qbittorrent-nox", "") != True:
    get_ipython().system_raw("qbittorrent-nox -d --webui-port=$Port_QB")
  if CheckProcess("autossh", Random_URL_QB) != True:
      AutoSSH(Random_URL_QB, Port_QB)

def Start_Localhost_QB():
  try:
    clear_output(wait=True)
    !autossh -l $Random_URL_QB -M 0 -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R 80:localhost:$Port_QB ssh.localhost.run
  except:
    Control_Panel_QB()
  Control_Panel_QB()

def Control_Panel_QB():
  clear_output(wait=True)
  display(MakeLabel("✔ Successfully", "success"), MakeButton("Recheck", Start_AutoSSH_QB), MakeButton("Backup Website", Start_Localhost_QB), HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">qBittorrent</h2><h4 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">" \
                    "<a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"http://" + Random_URL_QB + ".serveo.net\" target=\"_blank\">Website</a></h4>"))

try:
  try:
    Random_URL_QB
  except NameError:
    Random_URL_QB = "qb" + RandomGenerator()
    Port_QB = "6006"
    display(MakeLabel("Installing in Progress", "warning"))
    get_ipython().system_raw("rm -rf /content/sample_data/ &")
  if os.path.isfile("/usr/bin/autossh") == False:
    get_ipython().system_raw("apt update -qq -y && apt install autossh -qq -y")
  if os.path.isfile("/usr/bin/qbittorrent-nox") == False:
    if Old_Version:
      get_ipython().system_raw("apt install libtorrent-rasterbar9 -qq -y && wget -q https://launchpad.net/~qbittorrent-team/+archive/ubuntu/qbittorrent-stable/+build/17345008/+files/qbittorrent-nox_4.1.6.99~201907311051-6714-9c2a114~ubuntu18.04.1_amd64.deb -O /root/.qbittorrent-nox_4.1.6.deb && dpkg -i /root/.qbittorrent-nox_4.1.6.deb")
      # get_ipython().system_raw("apt install qbittorrent-nox=4.0.3-1 -qq -y")
    else:
      get_ipython().system_raw("yes "" | add-apt-repository ppa:qbittorrent-team/qbittorrent-stable && apt install qbittorrent-nox -qq -y")
    get_ipython().system_raw("mkdir -p -m 666 /{content/qBittorrent,root/{.qBittorrent_temp,.config/qBittorrent}} && wget -q https://raw.githubusercontent.com/Sparoney/RcloneLab/master/res/qbittorrent/qBittorrent.conf -O /root/.config/qBittorrent/qBittorrent.conf")
  Start_AutoSSH_QB()
  Control_Panel_QB()
except:
  clear_output(wait=True)
  display(MakeLabel("✘ Unsuccessfully", "danger"))


Welcome to localhost.run!

Head over to https://twitter.com/localhost_run and give us a follow for the
hottest SSH port forwarded local dev env news.

**You're going to need a SSH key to access this service soon.**

If you don't know what this is pls don't panic, you're smart, you've got this.
Github has a great howto, follow along with it to get prepared for the change:
https://help.github.com/en/github/authenticating-to-github/generating-a-new-ssh-key-and-adding-it-to-the-ssh-agent

Why is this happening you ask? Behind the scenes we've been hard at work.
There's bug fixes, stability improvements and soon, new features, some of which
require that SSH key we mentioned. Probably a new website too, for peeps who
don't yet know the joys of using SSH to read this. Watch this space...

**If you see a message below telling you that your hostname is already in use**
** try a different username like this:**
**`ssh -R localhost:80:localhost:8080 random-string@ssh.localhost.run`**

Connect to 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# <img src='https://raw.githubusercontent.com/Sparoney/RcloneLab/master/img/title_jdownloader.png' height="45" alt="JDownloader"/>

In [0]:
#@markdown <h3>⬅️ Click Here to Install JDownloader</h3>

import os, uuid, re, IPython
import ipywidgets as widgets

from glob import glob
from IPython.display import HTML, clear_output
from google.colab import output

Email = widgets.Text(placeholder="*Required", description="Email:")
Password = widgets.Text(placeholder="*Required", description="Password:")
Device = widgets.Text(placeholder="Optional", description="Name:")
SavePath = widgets.Dropdown(value="/content", options=["/content", "/content/Downloads"], description="Save Path:")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def RefreshPath():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePath.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePath.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePath.options = ["/content", "/content/Downloads"]

def LoginForm():
  clear_output(wait=True)
  Email.value = ""
  Password.value = ""
  Device.value = ""
  RefreshPath()
  display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">If you don't have an account yet, please register <a href=\"https://my.jdownloader.org/login.html#register\" target=\"_blank\">here</a>.</h3>"), HTML("<br>"), Email, Password, Device, SavePath, MakeButton("Refresh", RefreshPath, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML("*If you want to save in Google Drive please run the cell below."))
  display(HTML("<br>"), MakeButton("Login", CheckLogin, "info"))
  if os.path.isfile("/root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json"):
    display(MakeButton("Cancel", Show, "danger"))
  
def RestartForm():
  clear_output(wait=True)
  display(MakeLabel("Restart Confirm?", ""), MakeButton("Confirm", Restart, "danger"), MakeButton("Cancel", Show, "warning"))
  
def ExitForm():
  clear_output(wait=True)
  display(MakeLabel("Exit Confirm?", ""), MakeButton("Confirm", Exit, "danger"), MakeButton("Cancel", Show, "warning"))

def CheckLogin():
  try:
    if not Email.value.strip():
      ERROR = "Email field is empty."
      THROW_ERROR
    if not "@" in Email.value and not "." in Email.value:
      ERROR = "Email is an incorrect format."
      THROW_ERROR
    if not Password.value.strip():
      ERROR = "Password field is empty."
      THROW_ERROR
    if not bool(re.match("^[a-zA-Z0-9]+$", Device.value)) and Device.value.strip():
      ERROR = "Only alphanumeric are allowed for the device name."
      THROW_ERROR
    Login()
  except:
    print(ERROR)

def Login():
  clear_output(wait=True)
  if SavePath.value == "/content":
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"/content\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  elif SavePath.value == "/content/Downloads":
    get_ipython().system_raw("mkdir -p -m 666 /content/Downloads")
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"/content/Downloads\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  else:
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"" + SavePath.value + "\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  if Device.value.strip() == "":
    Device.value = Email.value
  get_ipython().system_raw("pkill -9 -e -f java")
  get_ipython().system_raw("echo '{\"email\" : \"'" + Email.value + "'\", \"password\" : \"'" + Password.value + "'\", \"devicename\" : \"'" + Device.value + "'\", \"directconnectmode\" : \"LAN\"}' > /root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json")
  get_ipython().system_raw("java -jar /root/.JDownloader/JDownloader.jar -norestart -noerr -r &")
  Show()

def Restart():
  get_ipython().system_raw("pkill -9 -e -f java")
  get_ipython().system_raw("java -jar /root/.JDownloader/JDownloader.jar -norestart -noerr -r &")
  Show()
  
def Exit():
  get_ipython().system_raw("pkill -9 -e -f java")
  clear_output(wait=True)
  display(MakeButton("Start", Restart, "info"))
  
def Show():
  clear_output(wait=True)
  display(MakeLabel("Control Panel", ""), HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">You can login to the WebUI by clicking <a href=\"https://my.jdownloader.org/\" target=\"_blank\">here</a>.</h3>"), HTML("<h4 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">If the server didn't showup in 30 sec. please re-login.</h4>"), HTML("<br>"), MakeButton("Re-Login", LoginForm, "info"), MakeButton("Restart", RestartForm, "warning"), MakeButton("Exit", ExitForm, "danger"))
  
    
if not os.path.isfile("/root/.JDownloader/JDownloader.jar"):
  clear_output(wait=True)
  display(MakeLabel("Installing in Progress", "warning"))
  get_ipython().system_raw("rm -rf /content/sample_data/ && apt install openjdk-8-jre-headless -qq -y && mkdir -p -m 666 /root/.JDownloader/libs && wget -q http://installer.jdownloader.org/JDownloader.jar -O /root/.JDownloader/JDownloader.jar && java -jar /root/.JDownloader/JDownloader.jar -norestart -h && wget -q https://raw.githubusercontent.com/Sparoney/RcloneLab/master/res/jdownloader/sevenzipjbinding1509.jar -O /root/.JDownloader/libs/sevenzipjbinding1509.jar && wget -q https://raw.githubusercontent.com/Sparoney/RcloneLab/master/res/jdownloader/sevenzipjbinding1509Linux.jar -O /root/.JDownloader/libs/sevenzipjbinding1509Linux.jar")
  LoginForm()
elif not os.path.isfile("/root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json"):
  LoginForm()
else:  
  Show()

Text(value='', description='Email:', placeholder='*Required')

Text(value='', description='Password:', placeholder='*Required')

Text(value='', description='Name:', placeholder='Optional')

Dropdown(description='Save Path:', options=('/content', '/content/Downloads'), value='/content')

In [0]:
#@markdown <h3>⬅️ Click Here to Mount Google Drive</h3>
#@markdown <h4>⚠️ If you download the file directly to Google Drive, it might have data corruption, use this at your own risk!</h4>
#@markdown <h4>✅ The rclone method is recommended and always the safest.</h4>

import os, uuid; import ipywidgets as widgets
from IPython.display import clear_output
from google.colab import drive, output

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)
    
def DemountForm():
  clear_output(wait=True)
  display(MakeLabel("Demount Confirm?", ""), MakeButton("Confirm", Demount, "danger"), MakeButton("Cancel", ShowGD, "warning"))
    
def Demount():
  clear_output(wait=True)
  get_ipython().system_raw("pkill -9 -e -f drive")
  !rm -rf /root/.config/Google/DriveFS/
  !echo "If you want to mount Google Drive run this cell again."
  
def ShowGD():
  try:
    RefreshPath()
  except:
    clear_output(wait=True)
  display(MakeLabel("Control Panel", ""), MakeButton("Demount", DemountForm, "danger"))

while not os.path.exists("/content/drive"):
  try:
    drive.mount("/content/drive")
    clear_output(wait=True)
  except:
    clear_output(wait=True)
    
ShowGD()

# <img src='https://raw.githubusercontent.com/Sparoney/RcloneLab/master/img/title_youtube-dl.png' height="45" alt="Youtube-DL"/>

In [0]:
#@markdown <h3>⬅️ Click Here to Use YouTube-DL</h3>
#@markdown <h4>📝 Note: if you want to change an archive file just run this cell again.</h4>
Archive = False #@param {type:"boolean"}

import os, uuid, urllib.parse
import ipywidgets as widgets

from glob import glob
from urllib.parse import urlparse, parse_qs
from IPython.display import HTML, clear_output, YouTubeVideo
from IPython.utils.io import ask_yes_no
from google.colab import output, files

Links = widgets.Textarea(placeholder='''Video/Playlist Link
(one link per line)''')

VideoQ = widgets.Dropdown(options=["Best Quality (VP9 upto 4K)", "Best Compatibility (H.264 upto 1080p)"])

AudioQ = widgets.Dropdown(options=["Best Quality (Opus)", "Best Compatibility (M4A)"])

Subtitle = widgets.ToggleButton(value=True, description="Subtitle", button_style="info", tooltip="Subtitle")

SavePathYT = widgets.Dropdown(options=["/content", "/content/Downloads"])

AudioOnly = widgets.ToggleButton(value=False, description="Audio Only", button_style="", tooltip="Audio Only")

Resolution = widgets.Select(options=["Highest", "4K", "1440p", "1080p", "720p", "480p", "360p", "240p", "144p"], value="Highest")

Extension = widgets.Select(options=["mkv", "webm"], value="mkv")

UsernameYT = widgets.Text(placeholder="Username")

PasswordYT = widgets.Text(placeholder="Password")

SecAuth = widgets.Text(placeholder="2nd Factor Authentication")

VideoPW = widgets.Text(placeholder="Video Password")

GEOBypass = widgets.Dropdown(options=["Disable", "Hide", "AD", "AE", "AF", "AG", "AI", "AL", "AM", "AO", "AQ", "AR", "AS", "AT", "AU", "AW", "AX", "AZ", "BA", "BB", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BL", "BM", "BN", "BO", "BQ", "BR", "BS", "BT", "BV", "BW", "BY", "BZ", "CA", "CC", "CD", "CF", "CG", "CH", "CI", "CK", "CL", "CM", "CN", "CO", "CR", "CU", "CV", "CW", "CX", "CY", "CZ", "DE", "DJ", "DK", "DM", "DO", "DZ", "EC", "EE", "EG", "EH", "ER", "ES", "ET", "FI", "FJ", "FK", "FM", "FO", "FR", "GA", "GB", "GD", "GE", "GF", "GG", "GH", "GI", "GL", "GM", "GN", "GP", "GQ", "GR", "GS", "GT", "GU", "GW", "GY", "HK", "HM", "HN", "HR", "HT", "HU", "ID", "IE", "IL", "IM", "IN", "IO", "IQ", "IR", "IS", "IT", "JE", "JM", "JO", "JP", "KE", "KG", "KH", "KI", "KM", "KN", "KP", "KR", "KW", "KY", "KZ", "LA", "LB", "LC", "LI", "LK", "LR", "LS", "LT", "LU", "LV", "LY", "MA", "MC", "MD", "ME", "MF", "MG", "MH", "MK", "ML", "MM", "MN", "MO", "MP", "MQ", "MR", "MS", "MT", "MU", "MV", "MW", "MX", "MY", "MZ", "NA", "NC", "NE", "NF", "NG", "NI", "NL", "NO", "NP", "NR", "NU", "NZ", "OM", "PA", "PE", "PF", "PG", "PH", "PK", "PL", "PM", "PN", "PR", "PS", "PT", "PW", "PY", "QA", "RE", "RO", "RS", "RU", "RW", "SA", "SB", "SC", "SD", "SE", "SG", "SH", "SI", "SJ", "SK", "SL", "SM", "SN", "SO", "SR", "SS", "ST", "SV", "SX", "SY", "SZ", "TC", "TD", "TF", "TG", "TH", "TJ", "TK", "TL", "TM", "TN", "TO", "TR", "TT", "TV", "TW", "TZ", "UA", "UG", "UM", "US", "UY", "UZ", "VA", "VC", "VE", "VG", "VI", "VN", "VU", "WF", "WS", "YE", "YT", "ZA", "ZM", "ZW"])

ProxyYT = widgets.Text(placeholder="Proxy URL")

MinSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Min:")

MaxSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Max:")

ExtraArg = widgets.Text(placeholder="Extra Arguments")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def upload_archive():
  if ask_yes_no("Do you already have an archive file? (y/n)", default="", interrupt=""):
    try:
      display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Please upload an archive from your computer.</h2><br>"))
      UploadConfig = files.upload().keys()
      clear_output(wait=True)
      if len(UploadConfig) == 0:
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
      elif len(UploadConfig) == 1:
        for fn in UploadConfig:
          if os.path.isfile("/content/" + fn):
            get_ipython().system_raw("mv -f " + "\"" + fn + "\" /root/.youtube-dl.txt && chmod 666 /root/.youtube-dl.txt")
            AudioOnly.observe(AudioOnlyChange)
            Subtitle.observe(SubtitleChange)
            AudioQ.observe(AudioQChange)
            ShowYT()
          else:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
      else:
        for fn in UploadConfig:
          get_ipython().system_raw("rm -f " + "\"" + fn + "\"")
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
    except:
      clear_output(wait=True)
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))
  else:
    get_ipython().system_raw("touch '/root/.youtube-dl.txt'")
    AudioOnly.observe(AudioOnlyChange)
    Subtitle.observe(SubtitleChange)
    AudioQ.observe(AudioQChange)
    ShowYT()

def RefreshPathYT():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePathYT.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePathYT.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePathYT.options = ["/content", "/content/Downloads"]

def AudioOnlyChange(change):
  if change["type"] == "change" and change["new"]:
    VideoQ.disabled = True
    Subtitle.disabled = True
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = True
    Extension.options = ["best", "aac", "flac", "mp3", "m4a", "opus", "vorbis", "wav"]
    Extension.value = "best"
    AudioOnly.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    VideoQ.disabled = False
    Subtitle.disabled = False
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = False
    if AudioQ.value == "Best Quality (Opus)":
      Extension.options = ["mkv", "webm"]
    else:
      Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"
    AudioOnly.button_style = ""

def SubtitleChange(change):
  if change["type"] == "change" and change["new"]:
    Subtitle.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    Subtitle.button_style = ""

def AudioQChange(change):
  if change["type"] == "change" and change["new"] == "Best Quality (Opus)":
    Extension.options = ["mkv", "webm"]
    Extension.value = "mkv"
  elif change["type"] == "change" and change["new"] == "Best Compatibility (M4A)":
    Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"

def ShowYT():
  clear_output(wait=True)
  RefreshPathYT()
  display(widgets.HBox([widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Link:</b>"), Links,
                                      widgets.HTML("<b style=\"color:#888888;\">For website that require an account:</b>"), UsernameYT, PasswordYT, SecAuth, VideoPW,
                                      widgets.HTML("<b><a href=\"https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2#Officially_assigned_code_elements\" target=\"_blank\">GEO Bypass Country:</a></b>"), GEOBypass,
                                      widgets.HTML("<b style=\"color:#888888;\">Proxy:</b>"), ProxyYT,
                                      widgets.HTML("<b style=\"color:#888888;\">Sleep Interval (second):</b>"), MinSleep, MaxSleep]),
                        widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Video Quality:</b>"), VideoQ, widgets.HTML("<b style=\"color:#888888;\">Resolution:</b>"), Resolution,
                                      widgets.HTML("<b style=\"color:#888888;\">Audio Quality:</b>"), AudioQ, widgets.HTML("<b style=\"color:#888888;\">Extension:</b>"), Extension,
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Options:</b>"), widgets.HBox([Subtitle, AudioOnly]),
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Arguments:</b>"), ExtraArg])]), HTML("<h4 style=\"color:#888888;\">Save Location:</h4>"),
          SavePathYT, MakeButton("Refresh", RefreshPathYT, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML("*If you want to save in Google Drive please run the cell below."))
  display(HTML("<br>"), MakeButton("Download", DownloadYT, "info"))

def DownloadYT():
  if Links.value.strip():
    Count = 0
    Total = str(len(Links.value.splitlines()))
    # Account Check
    if UsernameYT.value.strip() and PasswordYT.value.strip():
      accountC = "--username \"" + UsernameYT.value + "\" --password \"" + PasswordYT.value + "\""
    else:
      accountC = ""
    if SecAuth.value.strip():
      secauthC = "-2 " + SecAuth.value
    else:
      secauthC = ""
    if VideoPW.value.strip():
      videopwC = "--video-password " + VideoPW.value
    else:
      videopwC = ""
    # Proxy
    if ProxyYT.value.strip():
      proxyytC = "--proxy " + ProxyYT.value
    else:
      proxyytC = ""
    # GEO Bypass
    if GEOBypass.value == "Disable":
      geobypass = ""
    elif GEOBypass.value == "Hide":
      geobypass = "--geo-bypass"
    else:
      geobypass = "--geo-bypass-country " + GEOBypass.value
    # Video Quality
    if VideoQ.value == "Best Quality (VP9 upto 4K)":
      videoqC = "webm"
    else:
      videoqC = "mp4"
    # Audio Quality
    if AudioQ.value == "Best Quality (Opus)":
      audioqC = "webm"
    else:
      audioqC = "m4a"
    # Audio Only Check
    if AudioOnly.value:
      subtitleC = ""
      thumbnailC = ""
      extC = "-x --audio-quality 0 --audio-format " + Extension.value
      codecC = "bestaudio[ext=" + audioqC + "]/bestaudio/best"
    else:
      if Subtitle.value:
        subtitleC = "--all-subs --convert-subs srt --embed-subs"
      else:
        subtitleC = ""
      if Extension.value == "mp4":
        thumbnailC = "--embed-thumbnail"
      else:
        thumbnailC = ""
      extC = "--merge-output-format " + Extension.value
      if Resolution.value == "Highest":
        codecC = "bestvideo[ext=" + videoqC + "]+bestaudio[ext=" + audioqC + "]/bestvideo+bestaudio/best"
      else:
        codecC = "bestvideo[ext=" + videoqC + ",height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio[ext=" + audioqC + "]/bestvideo[height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio/bestvideo+bestaudio/best"
    # Archive
    if os.path.isfile("/root/.youtube-dl.txt"):
      archiveC = "--download-archive \"/root/.youtube-dl.txt\""
    else:
      archiveC = ""
    # Sleep Interval
    if MinSleep.value > 0 and MaxSleep.value > 0:
      minsleepC = "--min-sleep-interval " + MinSleep.value
      maxsleepC = "--max-sleep-interval " + MaxSleep.value
    else:
      minsleepC = ""
      maxsleepC = ""
    # Extra Arguments
    extraargC = ExtraArg.value
    for Link in Links.value.splitlines():
      clear_output(wait=True)
      Count += 1
      display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Processing link " + str(Count) + " out of " + Total + "</h3>"))
      if "youtube.com" in Link or "youtu.be" in Link:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading...</h3><br>"), YouTubeVideo(Link, width=640, height=360), HTML("<br>"))
      else:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading <a href=\"" + Link + "\">" + Link + "</a></h3><br>"))
      if ("youtube.com" in Link or "youtu.be" in Link) and "list=" in Link:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(playlist)s/%(playlist_index)s - %(title)s.%(ext)s" "$Link"
      else:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(title)s.%(ext)s" "$Link"
      if not os.path.exists(SavePathYT.value):
        get_ipython().system_raw("mkdir -p -m 666 " + SavePathYT.value)
      get_ipython().system_raw("mv /root/.YouTube-DL/* '" + SavePathYT.value + "/'")
    # Archive Download
    if os.path.isfile("/root/.youtube-dl.txt"):
      files.download("/root/.youtube-dl.txt")
    ShowYT()

if not os.path.isfile("/usr/local/bin/youtube-dl"):
  get_ipython().system_raw("rm -rf /content/sample_data/ && mkdir -p -m 666 /root/.YouTube-DL/ && apt-get install atomicparsley && curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl && chmod a+rx /usr/local/bin/youtube-dl")
if Archive:
  upload_archive()
else:
  AudioOnly.observe(AudioOnlyChange)
  Subtitle.observe(SubtitleChange)
  AudioQ.observe(AudioQChange)
  ShowYT()

Dropdown(options=('/content', '/content/Downloads'), value='/content')

In [0]:
#@markdown <h3>⬅️ Click Here to Mount Google Drive</h3>
#@markdown <h4>⚠️ If you download the file directly to Google Drive, it might have data corruption, use this at your own risk!</h4>
#@markdown <h4>✅ The rclone method is recommended and always the safest.</h4>

import os, uuid; import ipywidgets as widgets
from IPython.display import clear_output
from google.colab import drive, output

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)
    
def DemountForm():
  clear_output(wait=True)
  display(MakeLabel("Demount Confirm?", ""), MakeButton("Confirm", Demount, "danger"), MakeButton("Cancel", ShowGD, "warning"))
    
def Demount():
  clear_output(wait=True)
  get_ipython().system_raw("pkill -9 -e -f drive")
  !rm -rf /root/.config/Google/DriveFS/
  !echo "If you want to mount Google Drive run this cell again."
  
def ShowGD():
  try:
    RefreshPathYT()
  except:
    clear_output(wait=True)
  display(MakeLabel("Control Panel", ""), MakeButton("Demount", DemountForm, "danger"))

while not os.path.exists("/content/drive"):
  try:
    drive.mount("/content/drive")
    clear_output(wait=True)
  except:
    clear_output(wait=True)
    
ShowGD()

# <img src='https://raw.githubusercontent.com/Sparoney/RcloneLab/master/img/title_utility.png' height="45" alt="Utility"/>

In [0]:
#@markdown <h3>⬅️ Install Cloud Commander (file manager)</h3>
#@markdown <br><center><img src='https://raw.githubusercontent.com/Sparoney/RcloneLab/master/img/title_cloud_commander.png' height="60" alt="netdata"/></center>

import os, psutil, IPython, uuid, time
import ipywidgets as widgets

from IPython.display import HTML, clear_output
from google.colab import output

class MakeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    template = """<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-info" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def RandomGenerator():
  return time.strftime("%S") + str(time.time()).split(".")[-1]

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue
  
def AutoSSH(name,port):
  get_ipython().system_raw("autossh -M 0 -fNT -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R " + name + ":80:localhost:" + port + " serveo.net &")

def Start_AutoSSH_CC():
  if CheckProcess("cloudcmd", "") != True:
    get_ipython().system_raw("cloudcmd --online --no-auth --show-config --show-file-name --editor 'deepword' --packer 'tar' --port $Port_CC --progress --no-confirm-copy --confirm-move --name 'RcloneLab File Manager' --keys-panel --no-contact --console --sync-console-path --no-terminal --no-vim --columns 'name-size-date' --no-log &")
  if CheckProcess("autossh", Random_URL_CC) != True:
    AutoSSH(Random_URL_CC, Port_CC)

def Start_Localhost_CC():
  try:
    clear_output(wait=True)
    !autossh -l $Random_URL_CC -M 0 -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R 80:localhost:$Port_CC ssh.localhost.run
  except:
    Control_Panel_CC()
  Control_Panel_CC()

def Control_Panel_CC():
  clear_output(wait=True)
  display(MakeLabel("✔ Successfully", "success"), MakeButton("Recheck", Start_AutoSSH_CC), MakeButton("Backup Website", Start_Localhost_CC), HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">File Manager</h2><h4 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">" \
                    "<a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"https://" + Random_URL_CC + ".serveo.net/fs/content/\" target=\"_blank\">Website</a></h3>"))

try:
  try:
    Random_URL_CC
  except NameError:
    Random_URL_CC = "cc" + RandomGenerator()
    Port_CC = "7007"
    display(MakeLabel('Installing in Progress', 'warning'))
  if os.path.isfile("/tools/node/bin/cloudcmd") == False:
    get_ipython().system_raw("rm -rf /content/sample_data/ && npm i -g npm && npm i cloudcmd -g")
  if os.path.isfile("/usr/bin/autossh") == False:
    get_ipython().system_raw("apt update -qq -y && apt install autossh -qq -y")
  Start_AutoSSH_CC()
  Control_Panel_CC()
except:
  clear_output(wait=True)
  display(MakeLabel("✘ Unsuccessfully", "danger")) 

Button(button_style='success', description='✔ Successfully', disabled=True, style=ButtonStyle())

In [0]:
# ============================= FORM ============================= #
#@markdown <h3>⬅️ Install netdata (Real-time Server Monitoring)</h3>
#@markdown <br><center><img src='https://raw.githubusercontent.com/Sparoney/RcloneLab/master/img/title_netdata.png' height="60" alt="netdata"/></center>
# ================================================================ #

import os, psutil, IPython, uuid, time
import ipywidgets as widgets

from IPython.display import HTML, clear_output
from google.colab import output

class MakeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    template = """<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-info" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def RandomGenerator():
  return time.strftime("%S") + str(time.time()).split(".")[-1]

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue

def AutoSSH(name,port):
  get_ipython().system_raw("autossh -M 0 -fNT -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R " + name + ":80:localhost:" + port + " serveo.net &")

def Start_AutoSSH_ND():
  if CheckProcess("netdata", "") != True:
    get_ipython().system_raw("/usr/sbin/netdata")
  if CheckProcess("autossh", Random_URL_ND) != True:
      AutoSSH(Random_URL_ND, Port_ND)

def Start_Localhost_ND():
  try:
    clear_output(wait=True)
    !autossh -l $Random_URL_ND -M 0 -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R 80:localhost:$Port_ND ssh.localhost.run
  except:
    Control_Panel_CC()
  Control_Panel_CC()

def Control_Panel_ND():
  clear_output(wait=True)
  display(MakeLabel("✔ Successfully", "success"), MakeButton("Recheck", Start_AutoSSH_ND), MakeButton("Backup Website", Start_Localhost_ND), HTML("<h4 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">" \
                    "<a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"https://" + Random_URL_ND + ".serveo.net/#theme=slate;update_always=true\" target=\"_blank\">Website</a></h4>"))

try:
  try:
    Random_URL_ND
  except NameError:
    Random_URL_ND = "nd" + RandomGenerator()
    Port_ND = "19999"
    display(MakeLabel("Installing in Progress", "warning"))
  if os.path.isfile("/usr/bin/autossh") == False:
    get_ipython().system_raw("apt update -qq -y && apt install autossh -qq -y")
  if os.path.isfile("/usr/sbin/netdata") == False:
    get_ipython().system_raw("bash <(curl -Ss https://my-netdata.io/kickstart.sh) --dont-wait --dont-start-it")
  Start_AutoSSH_ND()
  Control_Panel_ND()
except:
  clear_output(wait=True)
  display(MakeLabel("✘ Unsuccessfully", "danger"))

Button(button_style='success', description='✔ Successfully', disabled=True, style=ButtonStyle())

In [0]:
# ============================= FORM ============================= #
#@markdown <h3>⬅️ Check VM's Status</h3>
Loop_Check = False #@param {type:"boolean"}
Loop_Interval = 5 #@param {type:"slider", min:1, max:15, step:1}
# ================================================================ #

import time
from IPython.display import clear_output
Loop = True

try:
  while Loop == True:
    clear_output(wait=True)
    !top -bcn1 -w512
    if Loop_Check == False:
      Loop = False
    else:
      time.sleep(Loop_Interval)
except:
  clear_output()

In [0]:
# ============================= FORM ============================= #
#@markdown <h3>⬅️ Get VM's Specification</h3>
Output_Format = "TEXT" #@param ["TEXT", "HTML", "XML", "JSON"]
Short_Output = True #@param {type:"boolean"}
# ================================================================ #

import os
from google.colab import files
from IPython.display import HTML, clear_output

try:
  Output_Format_Ext
except NameError:
  get_ipython().system_raw("apt install lshw -qq -y")

if Short_Output:
  Output_Format = "txt"
  Output_Format2 = "-short"
  Output_Format_Ext = "txt"
elif Output_Format == "TEXT":
  Output_Format = "txt"
  Output_Format2 = ""
  Output_Format_Ext = "txt"
else:
  Output_Format = Output_Format.lower()
  Output_Format2 = "-"+Output_Format.lower()
  Output_Format_Ext = Output_Format.lower()

get_ipython().system_raw("lshw " + Output_Format2 + " > Specification." + Output_Format)
files.download("/content/Specification." + Output_Format_Ext)
get_ipython().system_raw("rm -f /content/Specification.$outputformatC")
display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Sending log to your browser...</h2><br></center>"))